In [ ]:
#Import Libraries

import polars as pl
import pandas as pd
import streamlit as st
import numpy as np
import xlsx2csv
from lets_plot import *

In [ ]:
epds = pl.read_csv('2024epd.csv', ignore_errors=True).with_row_index().drop(['Sex', 'DOB','BrdCds'])

epds = epds.with_columns(mean = pl.sum_horizontal(['PROS','HB','GM']) / 3)


epds = (epds.with_columns([
    pl.col("AnimalID").str.split_exact(" - ", 1).
    alias("split_column")]).
    with_columns(pl.col('split_column')).
    unnest('split_column').
    rename({'field_0' : 'cow', 'field_1' : 'bull'})
)

criteria = {
    "ME": -1,
    "CED": 12,
    "Milk": 26,
    "CEM": 7,
    "Marb": 0.4,
    "STAY": 17
}

epds_mins = epds.filter(
    (pl.col("ME") <= criteria["ME"]) &
    (pl.col("CED") >= criteria["CED"]) &
    (pl.col("Milk") >= criteria["Milk"]) &
    (pl.col("CEM") >= criteria["CEM"]) &
    (pl.col("Marb") >= criteria["Marb"]) &
    (pl.col("STAY") >= criteria["STAY"])
)

6 cows who cannot meet milk requirement at all: C29, D10, G07, H15, H11, H59

4 cows cannot meet marb requirements at all: C44, F83, F03, C32

17 other cows that cannot meet the requirements in conjuction


In [ ]:
#epds.select(pl.col('cow').n_unique()).item()
epds_mins.select(pl.col('cow').n_unique()).item()

In [ ]:
cows = epds.select(pl.col('cow').unique()).to_series().to_list()

cows_meeting_requirements = epds_mins.select(pl.col('cow').unique()).to_series().to_list()

# Compare the lists
cows_in_both = set(cows).intersection(cows_meeting_requirements)

cows_only_in_original = set(cows) - set(cows_meeting_requirements)

In [ ]:
cows_failing = epds.filter(pl.col("cow").is_in(cows_only_in_original))

In [ ]:
ggplot()